In [23]:
import pynucastro as pyna
reaclibrary = pyna.ReacLibLibrary()
all_nuclei = ["p","n", "h2"]#, "h3", "he3", "he4","Li6","Li7","Be7","Li8","B8","Be9","B10","B11","C11","B12","C12","N12","C13","N13","C14","N14","O14","N15","O15","O16"]
bbn_library = reaclibrary.linking_nuclei(all_nuclei)
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)


In [24]:
p__n_string = '''
b0 = -0.62173 
b1 = 0.22211e2
b2 = -0.72798e2
b3 = 0.11571e3
b4 = -0.11763e2
b5 = 0.45521e2
b6 = -3.7973 
b7 = 0.41266 
b8 = -0.026210
b9 = 0.87934e-3
b10 = -0.12016e-4
qpn = 2.8602

@numba.njit() 
def p__n(rate_eval, tf):  
    # p --> n
    z=5.92989658*tf.T9i
    rate=0
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    b=[b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10]
    if tf.T9>1.160451812:
      for i in range(11):
         rate+=1/880.2*np.exp(-qpn*z)*b[i]*z**-i 
        
    #Kawano rate
    #rate=1/879.6*(5.252/z - 16.229/z**2 + 18.059/z**3 + 34.181/z**4 + 27.617/z**5)*np.exp(-2.530988*z)
    rate_eval.p__n = rate

'''

In [25]:
n__p_string = '''
a0 = 1
a1 = 0.15735 
a2 = 4.6172
a3 = -0.40520e2 
a4 = 0.13875e3 
a5 = -0.59898e2
a6 = 0.66752e2 
a7 = -0.16705e2 
a8 = 3.8071
a9 = -0.39140 
a10 = 0.023590 
a11 = -0.83696e-4
a12 = -0.42095e-4 
a13 = 0.17675e-5
qnp = 0.33979 

@numba.njit()
def n__p(rate_eval, tf):
    # n --> p
    z=5.92989658*tf.T9i
    #rate from https://arxiv.org/pdf/astro-ph/0408076.pdf appendix C
    rate=0
    a=[a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13]
    for i in range(14):
      rate+=1/880.2*np.exp(-qnp/z)*a[i]*z**-i

    #Kawano rate
    #rate = 1/879.6*(0.565/z - 6.382/z**2 + 11.108/z**3 + 36.492/z**4 + 27.512/z**5)

    rate_eval.n__p = rate

'''

In [26]:
class n__p_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"
        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return n__p_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [27]:
n__p = n__p_Rate(reactants=[pyna.Nucleus("n")],
                  products=[pyna.Nucleus("p")],
                  r0=1, T0=1, nu=1)

In [28]:
class p__n_Rate(pyna.Rate):
    def __init__(self, reactants=None, products=None,
                 r0=1.0, T0=1.0, nu=0):

        # we'll take the Q value just to be the change in binding energy
        Q = 0
        for n in reactants:
            Q += -n.A * n.nucbind
        for n in products:
            Q += n.A * n.nucbind

        # we set the chapter to custom so the network knows how to deal with it
        self.chapter = "custom"

        self.reverse = None
    
        # call the Rate init to do the remaining initialization
        super().__init__(reactants=reactants, products=products, Q=Q)

        self.r0 = r0
        self.T0 = T0
        self.nu = nu

    def function_string_py(self):
        """return a string containing a python function that computes
        the rate"""
        return p__n_string

    def eval(self, T, rhoY=None):
        return self.r0 * (T / self.T0)**self.nu

In [29]:
p__n = p__n_Rate(reactants=[pyna.Nucleus("p")],
                  products=[pyna.Nucleus("n")],
                  r0=1, T0=1, nu=1)

In [30]:
p__n.fname='p__n'
n__p.fname='n__p'
#print(p__n.function_string_py())

In [31]:
bbn_network.validate(reaclibrary)

validation: d produced in n + p ⟶ H2 + 𝛾 never consumed.
validation: d produced in n + p + p ⟶ p + H2 never consumed.
validation: d produced in p + p + e⁻ ⟶ H2 + 𝜈 never consumed.
validation: d produced in p + p ⟶ H2 + e⁺ + 𝜈 never consumed.


False

In [32]:
class RatePair:
    """the forward and reverse rates for a single reaction sequence.
    Forward rates are those with Q >= 0.

    :var forward: the forward reaction Rate object
    :var reverse: the reverse reaction Rate object

    """

    def __init__(self, forward=None, reverse=None):
        self.forward = forward
        self.reverse = reverse

    def __repr__(self):
        return f"forward: {self.forward} ; reverse: {self.reverse}"

    def __lt__(self, other):
        if self.forward is not None and other.forward is not None:
            return self.forward < other.forward
        if self.forward is None:
            return False
        return True

    def __eq__(self, other):
        return self.forward == other.forward and self.reverse == other.reverse


In [33]:
bothrates=RatePair(p__n,n__p)

In [34]:
bothrates

forward: p ⟶ n + e⁺ + 𝜈 ; reverse: n ⟶ p + e⁻ + 𝜈

In [35]:
bbn_library += pyna.Library(rates=[p__n,n__p])
bbn_library.remove_rate(bbn_library.get_rate('n__p__weak__wc12'))

In [36]:
bbn_network = pyna.networks.PythonNetwork(libraries=bbn_library)

In [37]:
bbn_network.find_duplicate_links()

[[p + p ⟶ H2 + e⁺ + 𝜈, p + p + e⁻ ⟶ H2 + 𝜈]]

In [38]:
networkname='H2_net.py'
bbn_network.write_network(networkname)

file=open(networkname, 'a')
file.write('''
#For AoT compilation of the network
def AoT(networkname):
   from numba.pycc import CC

   cc = CC(networkname)
   # Uncomment the following line to print out the compilation steps
   #cc.verbose = True

   #
   @cc.export('nnuc','i4()')
   def nNuc():
      return nnuc

   @cc.export('rhs', 'f8[:](f8, f8[:], f8, f8)')
   def rhsCC(t, Y, rho, T):
      return rhs_eq(t, Y, rho, T, None)

   @cc.export('jacobian', '(f8, f8[:], f8, f8)')
   def jacobian(t, Y, rho, T):
      return jacobian_eq(t, Y, rho, T, None)


   cc.compile()
''') # Write some text
file.close() # Close the file


import H2_net as H2_n
H2_n.AoT('H2_AoT_net')


/home/hansbdein/Speciale/H2_net.py:372: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC


In [39]:
bbn_network.validate(reaclibrary)

validation: d produced in n + p ⟶ H2 + 𝛾 never consumed.
validation: d produced in n + p + p ⟶ p + H2 never consumed.
validation: d produced in p + p + e⁻ ⟶ H2 + 𝜈 never consumed.
validation: d produced in p + p ⟶ H2 + e⁺ + 𝜈 never consumed.


False

In [40]:
print(bbn_network.network_overview())

n
  consumed by:
     n + p ⟶ H2 + 𝛾
     n + p + p ⟶ p + H2
     n ⟶ p + e⁻ + 𝜈
  produced by:
     H2 ⟶ n + p
     H2 + p ⟶ n + p + p
     p ⟶ n + e⁺ + 𝜈

p
  consumed by:
     n + p ⟶ H2 + 𝛾
     p + p ⟶ H2 + e⁺ + 𝜈
     p + p + e⁻ ⟶ H2 + 𝜈
     H2 + p ⟶ n + p + p
     n + p + p ⟶ p + H2
     p ⟶ n + e⁺ + 𝜈
  produced by:
     H2 ⟶ n + p
     H2 + p ⟶ n + p + p
     n + p + p ⟶ p + H2
     n ⟶ p + e⁻ + 𝜈

d
  consumed by:
     H2 ⟶ n + p
     H2 + p ⟶ n + p + p
  produced by:
     n + p ⟶ H2 + 𝛾
     p + p ⟶ H2 + e⁺ + 𝜈
     p + p + e⁻ ⟶ H2 + 𝜈
     n + p + p ⟶ p + H2




In [41]:
filter = pyna.RateFilter(products=["He4"], exact=False)
He4rates = bbn_library.filter(filter).get_rates()
He4rates

AttributeError: 'NoneType' object has no attribute 'get_rates'

In [ ]:
TMeV2T9=11.60451812
Tpeak=0.5256676288178614*TMeV2T9

for rate in He4rates:
    print(rate.fname+'\t'+str(rate.eval(Tpeak)))


d_d__he4	0.0
p_t__he4	0.0
n_he3__he4	248.91468501748517
p_he3__he4__weak__bet_pos_	0.0
d_t__n_he4	0.0
d_he3__p_he4	0.0
t_he3__d_he4	0.0
t_t__n_n_he4	0.0
t_he3__n_p_he4	0.0
he3_he3__p_p_he4	0.0


In [ ]:
rate

sorted(He4rates,key=lambda rate: rate.eval(Tpeak),reverse=True)

[He3 + n ⟶ He4 + 𝛾,
 H2 + H2 ⟶ He4 + 𝛾,
 H3 + p ⟶ He4 + 𝛾,
 He3 + p ⟶ He4 + e⁺ + 𝜈,
 H3 + H2 ⟶ n + He4,
 He3 + H2 ⟶ p + He4,
 He3 + H3 ⟶ H2 + He4,
 H3 + H3 ⟶ n + n + He4,
 He3 + H3 ⟶ n + p + He4,
 He3 + He3 ⟶ p + p + He4]

In [ ]:
#For AoT compilation of the network
'''
from numba.pycc import CC

cc = CC('AoT_net')
# Uncomment the following line to print out the compilation steps
#cc.verbose = True

@cc.export('nnuc','i4()')
def nNuc():
    return nnuc

@cc.export('rhs', 'f8[:](f8, f8[:], f8, f8)')
def rhsCC(t, Y, rho, T):
    return rhs_eq(t, Y, rho, T, None)

@cc.export('jacobian', '(f8, f8[:], f8, f8)')
def jacobian(t, Y, rho, T):
    return jacobian_eq(t, Y, rho, T, None)


if __name__ == "__main__":
    cc.compile()

    

import He4_net as He4_n
importlib.reload(He4_n)
He4_n.AoT('He4_AoT_net')

'''

'\nfrom numba.pycc import CC\n\ncc = CC(\'AoT_net\')\n# Uncomment the following line to print out the compilation steps\n#cc.verbose = True\n\n@cc.export(\'nnuc\',\'i4()\')\ndef nNuc():\n    return nnuc\n\n@cc.export(\'rhs\', \'f8[:](f8, f8[:], f8, f8)\')\ndef rhsCC(t, Y, rho, T):\n    return rhs_eq(t, Y, rho, T, None)\n\n@cc.export(\'jacobian\', \'(f8, f8[:], f8, f8)\')\ndef jacobian(t, Y, rho, T):\n    return jacobian_eq(t, Y, rho, T, None)\n\n\nif __name__ == "__main__":\n    cc.compile()\n\n'